### Gold Layer

In [0]:
from pyspark.sql.functions import year, col

# Caminho da camada Silver
silver_directory_delta = "/covid/silver"

# Caminho da camada Gold
gold_directory_delta = "/covid/gold"

# Carregar os dados da camada Silver em formato Delta
df_silver = spark.read.format("delta").load(silver_directory_delta)


In [0]:
# 2. Adicionar coluna de ano para particionamento
df_silver_with_year = df_silver.withColumn("year", year(col("date")))

In [0]:
# Renomeando colunas
df_gold = df_silver_with_year.withColumnRenamed("new_confirmed", "daily_new_cases") \
                   .withColumnRenamed("new_deceased", "daily_new_deaths") \
                   .withColumnRenamed("cumulative_confirmed", "total_confirmed_cases") \
                   .withColumnRenamed("cumulative_deceased", "total_confirmed_deaths")


In [0]:
# 3. Escrever os dados na camada Gold, particionando por ano e aplicando ZORDER
(df_gold.write
 .format("delta")
 .mode("overwrite")
 .partitionBy("year")
 .option("overwriteSchema", "true")
 .save(gold_directory_delta))

# 4. Aplicar ZORDER BY na coluna 'location_key' para otimizar consultas
spark.sql(f"OPTIMIZE '{gold_directory_delta}' ZORDER BY (date)")

print(f"Dados particionados por ano e salvos na camada Gold: {gold_directory_delta}")

Dados particionados por ano e salvos na camada Gold: /covid/gold
